In [10]:
import os
from pathlib import Path
import obsidiantools.api as otools

# Set the path to the Obsidian vault
VAULT_PATH = Path(r'C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup')

# Verify the path exists
if VAULT_PATH.exists():
    print("Vault path exists.")
else:
    print("Vault path does not exist.")
    raise SystemExit("Exiting script due to invalid path.")

# Change the working directory to the vault path
os.chdir(VAULT_PATH)

# List the contents of the vault directory to check if it's correctly structured
print(f"Contents of the vault directory: {list(VAULT_PATH.iterdir())}")

# Connect to the vault
try:
    vault = otools.Vault(str(VAULT_PATH)).connect()
    print("Connected to the vault.")
except Exception as e:
    print(f"Failed to connect to the vault: {e}")
    raise

# Override _dirpath attribute with pathlib.Path object
vault._dirpath = Path(vault._dirpath)

# Gather the vault's metadata
try:
    vault.gather()
    print("Gathered vault metadata.")
except Exception as e:
    print(f"Failed to gather vault metadata: {e}")
    raise

# List all notes in the vault using the correct method
try:
    notes_metadata = vault.get_note_metadata()
    print(f"Type of notes_metadata: {type(notes_metadata)}")
    print(f"Content of notes_metadata: {notes_metadata.head()}")  # Print the first 5 items

    # Extract note names from the DataFrame index
    all_notes = notes_metadata.index.tolist()
    print(f"Total notes found: {len(all_notes)}")
    print(all_notes)
except Exception as e:
    print(f"Failed to list notes: {e}")
    raise

Vault path exists.
Contents of the vault directory: [WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/.git'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/.ipynb_checkpoints'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/.obsidian'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/.smart-connections'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/Daily Notes'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/Inbox'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/JupyterNotebooks'), WindowsPath('C:/Users/erinh.HYPERVSERVER/OneDrive/ObsidianVaults/Development/GroundUpRestart - Backup/metadataMKD.md'), Wind

In [23]:
from pathlib import Path
import obsidiantools.api as otools
import yaml

def identify_unfilled_metadata(vault_path, required_fields):
    # Initialize the vault
    vault = otools.Vault(str(vault_path))
    
    # Set the _dirpath attribute to a Path object
    vault._dirpath = Path(vault_path)
    
    vault.connect()
    vault.gather()
    notes_with_unfilled_metadata = []

    # Get note metadata
    notes_metadata = vault.get_note_metadata()

    for note_path in notes_metadata.index:
        note_file_path = vault._dirpath / note_path
        if note_file_path.exists():
            with open(note_file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                if content.startswith('---'):
                    try:
                        front_matter_end = content.find('---', 3)
                        yaml_content = content[:front_matter_end + 3]
                        metadata = yaml.safe_load(yaml_content.strip('---'))
                        
                        # Check if required metadata fields are empty or 'nan'
                        unfilled_metadata = {}
                        for field in required_fields:
                            value = metadata.get(field, None)
                            if value is None or (isinstance(value, str) and value.lower() == 'nan'):
                                unfilled_metadata[field] = value
                        
                        if unfilled_metadata:
                            notes_with_unfilled_metadata.append({
                                "note": note_path,
                                "unfilled_metadata": unfilled_metadata
                            })
                    except yaml.YAMLError as e:
                        print(f"Error parsing YAML for note {note_path}: {e}")

    return notes_with_unfilled_metadata

def main():
    vault_path = Path(r'C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup')
    required_fields = ['title', 'tags', 'description', 'parent_moc', 'related_notes', 'created', 'updated', 'author', 'status', 'priority']
    unfilled_metadata_notes = identify_unfilled_metadata(vault_path, required_fields)
    
    if unfilled_metadata_notes:
        print("Notes with unfilled metadata:")
        for item in unfilled_metadata_notes:
            print(f"Note: {item['note']}")
            for key, value in item['unfilled_metadata'].items():
                print(f"  Unfilled metadata: {key} (value: {value})")
    else:
        print("All required metadata is filled.")

if __name__ == "__main__":
    main()

All required metadata is filled.


In [30]:
#Export file metadata from folder
import os
import frontmatter  # Ensure you have the 'python-frontmatter' package installed
import csv

# Define the path to the Inbox folder
inbox_folder = r'C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup\Inbox'

# Define the path to the output CSV file
output_csv = r'C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup\metadata_output.csv'

# Function to process a note file
def process_note_file(note_path):
    try:
        with open(note_path, 'r', encoding='utf-8') as file:
            content = frontmatter.load(file)
            metadata = content.metadata
            return metadata
    except FileNotFoundError:
        print(f"Note file does not exist: {note_path}")
        return None
    except Exception as e:
        print(f"An error occurred while processing {note_path}: {e}")
        return None

# Collect metadata from all note files
all_metadata = []
for root, dirs, files in os.walk(inbox_folder):
    for file in files:
        note_path = os.path.join(root, file)
        metadata = process_note_file(note_path)
        if metadata is not None:
            metadata['filename'] = file  # Add filename to metadata
            all_metadata.append(metadata)

# Determine CSV headers from all collected metadata
headers = set()
for metadata in all_metadata:
    headers.update(metadata.keys())

# Write metadata to CSV
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    for metadata in all_metadata:
        writer.writerow(metadata)

print(f"Metadata has been written to {output_csv}")

Metadata has been written to C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup\metadata_output.csv


In [31]:
#GetMetadataFieldNames from files in a folder.
import os
import frontmatter  # Ensure you have the 'python-frontmatter' package installed
import csv

# Define the path to the Inbox folder
inbox_folder = r'C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup\Inbox'

# Define the path to the output CSV file
output_csv = r'C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup\metadata_field_names.csv'

# Function to process a note file
def process_note_file(note_path):
    try:
        with open(note_path, 'r', encoding='utf-8') as file:
            content = frontmatter.load(file)
            metadata = content.metadata
            return metadata.keys()
    except FileNotFoundError:
        print(f"Note file does not exist: {note_path}")
        return None
    except Exception as e:
        print(f"An error occurred while processing {note_path}: {e}")
        return None

# Collect metadata field names from all note files
all_metadata_field_names = []
for root, dirs, files in os.walk(inbox_folder):
    for file in files:
        note_path = os.path.join(root, file)
        metadata_field_names = process_note_file(note_path)
        if metadata_field_names is not None:
            all_metadata_field_names.append({
                'filename': file,
                'field_names': list(metadata_field_names)
            })

# Write metadata field names to CSV
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Filename', 'Metadata Field Names'])
    for item in all_metadata_field_names:
        writer.writerow([item['filename'], ', '.join(item['field_names'])])

print(f"Metadata field names have been written to {output_csv}")

Metadata field names have been written to C:\Users\erinh.HYPERVSERVER\OneDrive\ObsidianVaults\Development\GroundUpRestart - Backup\metadata_field_names.csv
